# 此為迴歸(Regression)演算法範例
- 如: 根據先前資料(氣象資料), 進行天氣預測明日溫度; 或依據軟體開發規格, 預測軟體專案開發的所需時間. 
- Regression(迴歸) => 迴歸演算法 不是 Logistic Regression(邏輯斯迴歸)=>分類演算法 

###  本例題僅有 506筆 data
- training data => 404, validation data => 102
- 每個 Feature 都擁有不同的刻度, 如: 介於 0~1 之間, 1~12 之間, 亦或是 0~100 之間

In [1]:
from keras.datasets import boston_housing
from keras import models, layers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

### Check Data.Shape

In [3]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


## 準備資料 
- 需先對 Data 進行"正規化(normalization)"處理

In [4]:
## 沿著第0軸(batch_size 軸), 做平均
mean = train_data.mean(axis = 0)
train_data -= mean

## 沿著第0軸(batch_size 軸), 算標準差
std  = train_data.std(axis = 0) 
train_data /= std

test_data -= mean
test_data /= std

## 建立神經網絡
- 2 個隱藏層, 各有 64 個神經元
- 因為training data 過少, 所以可能會有 overfitting 的問題產生
- 小型神經網絡, 是為了緩解 OverFitting 的問題產生

In [5]:
## import keras, layers, models......
from keras import layers, models

def build_model():
    input_shape = (train_data.shape[0], )
    model = models.Sequential()
    model.add(layers.Dense(64, 
                           activation = 'relu', 
                           input_shape = input_shape))
    model.add(layers.Dense(64, 
                           activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', 
                  loss='mse', 
                  metrics=['mae'])
    return model
    

In [6]:
## import numpy as np 
import numpy as np

In [7]:
k = 4 
num_val_samples = len(train_data) // k
num_epochs      = 100 
all_scores      = []

In [8]:
for i in range(k):
    print('Processing Flod #', i)
    val_data    = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], 
                                         train_data[(i + 1) * num_val_samples:]], axis = 0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], 
                                            train_targets[(i + 1) * num_val_samples:]], axis = 0)
    
    model = build_model()
    model.fit(partial_train_data, 
              partial_train_targets, 
              epochs = num_epochs, 
              batch_size = 1, 
              verbose = 0)
    
    val_mse, val_mae = model.evaluate(val_data, 
                                      val_targets, 
                                      verbose = 0)
    all_scores.append(val_mse)

W1223 10:55:40.483541 17308 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1223 10:55:40.512803 17308 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1223 10:55:40.519445 17308 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1223 10:55:40.550362 17308 deprecation_wrapper.py:119] From c:\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Processing Flod # 0


ValueError: Error when checking input: expected dense_1_input to have shape (404,) but got array with shape (13,)